# Convolution / Deconvolution
---
**Notebook 6 — 2026**
<div style="font-size:15px; padding:8px; margin:2px; font-weight:600; background-color:#E80808; color:white;text-align:center;">
    <div style=" ">
        Introduction to Multidimensional Fourier Transform
    </div>
</div>
<div style="border-bottom: 1px gray dotted; padding:8px;margin:2px;text-align:center; font-size:15px; color:#444;">
<i>Daniel Sage — École Polytechnique Fédérale de Lausanne (EPFL)</i>
</div>

**Important notice**: These interactive notebooks complement the lecture and are not self-contained; in-class explanations are required.

In [4]:
import numpy as np, matplotlib.pyplot as plt
from skimage import data, transform, io
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown
from matplotlib.patches import Circle
from skimage.filters import gaussian
from skimage.transform import rotate

#img = data.camera()
img = io.imread('car_pad.tif')
size=128
img = transform.resize(img, (size,size), anti_aliasing=True)

def fft2c(x): return np.fft.fftshift(np.fft.fft2(x))
def ifft2c(F): return np.real(np.fft.ifft2(np.fft.ifftshift(F)))
def snr(x,y): return 10*np.log10(np.mean(x*x)/(np.mean((x-y)**2)+1e-12))
def radial_grid(n):a=(np.arange(n)-n//2)/n; X,Y=np.meshgrid(a,a); return np.sqrt(X*X+Y*Y)
def showim(ax,img,t,vmin=None,vmax=None):
    ax.imshow(img,cmap="gray",vmin=vmin,vmax=vmax); ax.set_title(t,fontsize=16); ax.axis("off")
def showFM(ax,F,t):
    ax.imshow(np.log(np.abs(F)+1e-12),cmap="gray"); ax.set_title(t,fontsize=16); ax.axis("off")

## 5.1 Convolution / Multiplication

In [9]:
@interact(ker_name=Dropdown(options=["gaussian","disk","dx","dy"], value="gaussian"))
def conv_theorem(ker_name):
    I = img
    y,x = np.meshgrid(np.arange(size)-size//2, np.arange(size)-size//2, indexing="ij")
    if ker_name=="gaussian": sigma=1.5; K = np.exp(-np.sqrt(x*x + y*y)**2/(2*sigma**2))
    elif ker_name=="disk": sigma=3; K = (np.sqrt(x*x + y*y) <= sigma).astype(float)
    elif ker_name=="dx": K = np.zeros_like(I); K[size//2, size//2+1] = 1; K[size//2, size//2-1] = -1
    else: K = np.zeros_like(I); K[size//2+1, size//2] = 1; K[size//2-1, size//2] = -1

    K /= np.sum(np.abs(K)) + 1e-12
    FI = fft2c(I); FK = fft2c(np.fft.ifftshift(K))
    Fconv = FI * FK; Iconv = ifft2c(Fconv)
    Iref = ifft2c(fft2c(I) * fft2c(np.fft.ifftshift(K)))
    diff = np.linalg.norm(Iconv - Iref)

    fig,ax = plt.subplots(2,4,figsize=(16,8))
    showim(ax[0,0], I, "Image (I)");  showFM(ax[0,1], FI, "FFT(I)")
    showim(ax[0,2], K, "Kernel (K)"); showFM(ax[0,3], FK, "FFT(K)")
    showim(ax[1,0], Iconv, "Convolution(I,K)"); showFM(ax[1,1], Fconv, "FFT(I)·FFT(K)")
    showim(ax[1,2], Iref, "IFFT(FFT(I)·FFT(K))")
    showim(ax[1,3], Iconv - Iref, f"Diff ‖·‖={diff:.3e}")
    plt.show()

interactive(children=(Dropdown(description='ker_name', options=('gaussian', 'disk', 'dx', 'dy'), value='gaussi…

## 5.2 Convolution / Deconvolution

In [34]:
y,x = np.meshgrid(np.arange(size)-size//2, np.arange(size)-size//2, indexing="ij")

@interact(
    blur_sigma=FloatSlider(min=0.1,max=5,step=0.1,value=1),
    noise=FloatSlider(min=0.0,max=0.2,step=0.01,value=0.05),
    regul=FloatSlider(min=1e-4,max=10,step=1e-2,value=1e-2,readout_format=".4f")
)
def demo(blur_sigma, noise, regul):
    K = np.exp(-(x*x+y*y)/(2*blur_sigma**2))
    K /= K.sum()
    H = fft2c(np.fft.ifftshift(K))
    blur = ifft2c(fft2c(img)*H)

    noisy = blur + noise*np.random.randn(*blur.shape)
    F = fft2c(noisy)
    deconv = ifft2c(F*np.conj(H)/(np.abs(H)**2+eps))

    fig,ax=plt.subplots(2,4,figsize=(12,6))
    showim(ax[0,0],img,f"original \nSNR={snr(img,img):.2f} dB")
    showim(ax[0,1],blur,f"blur \nSNR={snr(img,blur):.2f} dB")
    showim(ax[0,2],noisy,f"blur + noise \nSNR={snr(img,noisy):.2f} dB")
    showim(ax[0,3],deconv,f"deconv \nSNR={snr(img,deconv):.2f} dB")

    showFM(ax[1,0],fft2c(img),"")
    showFM(ax[1,1],fft2c(blur),"")
    showFM(ax[1,2],fft2c(noisy),"")
    showFM(ax[1,3],fft2c(deconv),"")
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='blur_sigma', max=5.0, min=0.1), FloatSlider(value=0.…